In [ ]:
import osmnx as ox
import geopandas as gpd
import networkx as nx
from shapely import Point, LineString, Polygon
from quackosm import convert_geometry_to_geodataframe, geocode_to_geometry
import numpy as np
from pyproj import Transformer
from shapely.ops import transform
from matplotlib import pyplot as plt
import contextily as cx

In [ ]:
g_walk = ox.load_graphml("wroclaw_walk.graphml")

In [ ]:
_center_point = Point(17.03291465713426, 51.10909801275284)
center_node = ox.nearest_nodes(g_walk, X=_center_point.x, Y=_center_point.y)
center_point = Point(g_walk.nodes[center_node]["x"], g_walk.nodes[center_node]["y"])

In [ ]:
transformer_4326_2180 = Transformer.from_crs(4326, 2180, always_xy=True)
transformer_2180_4326 = Transformer.from_crs(2180, 4326, always_xy=True)

In [ ]:
# 500 meters

In [ ]:
distance_meters = 500

buffered_polygon_2180 = transform(transformer_4326_2180.transform, center_point).buffer(
    distance_meters
)
buffered_polygon_4326 = transform(
    transformer_2180_4326.transform, buffered_polygon_2180
)

clipped_graph_by_polygon = ox.truncate.truncate_graph_polygon(
    G=g_walk, polygon=buffered_polygon_4326, truncate_by_edge=True
)

clipped_graph_by_distance = ox.truncate.truncate_graph_dist(
    G=g_walk, source_node=center_node, dist=distance_meters
)

In [ ]:
from contextlib import suppress


with suppress(ValueError):
    clipped_graph_by_polygon_edges = ox.graph_to_gdfs(
        clipped_graph_by_polygon, nodes=False, edges=True
    )
    clipped_graph_by_distance_edges = ox.graph_to_gdfs(
        clipped_graph_by_distance, nodes=False, edges=True
    )

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(25, 10))

clipped_graph_by_polygon_edges.plot(ax=ax[0], lw=1, zorder=1)
gpd.GeoSeries([buffered_polygon_4326], crs=4326).boundary.plot(
    ax=ax[0], lw=1.5, color="black", zorder=2, ls="--", alpha=0.4
)

clipped_graph_by_distance_edges.plot(ax=ax[1], lw=1, zorder=1)
gpd.GeoSeries([buffered_polygon_4326], crs=4326).boundary.plot(
    ax=ax[1], lw=1.5, color="black", zorder=2, ls="--", alpha=0.4
)

clipped_graph_by_polygon_edges.plot(ax=ax[2], alpha=0.4, lw=1, ls="--", zorder=1)
clipped_graph_by_distance_edges.plot(ax=ax[2], color="orange", lw=1.5, zorder=2)

bounds = list(buffered_polygon_4326.bounds)
for _ax in ax:
    bounds_x = _ax.get_xlim()
    bounds_y = _ax.get_ylim()
    bounds[0] = min(bounds[0], bounds_x[0])
    bounds[1] = min(bounds[1], bounds_y[0])
    bounds[2] = max(bounds[2], bounds_x[1])
    bounds[3] = max(bounds[3], bounds_y[1])

for _ax in ax:
    # gpd.GeoSeries([center_point], crs=4326).plot(ax=_ax, color="black", zorder=3)
    gpd.GeoSeries([center_point], crs=4326).plot(
        ax=_ax,
        facecolor="white",
        edgecolor="black",
        zorder=3,
        markersize=100,
        lw=2,
        # edgewidth=1,
    )
    _ax.set_xlim(bounds[0], bounds[2])
    _ax.set_ylim(bounds[1], bounds[3])
    _ax.set_axis_off()
    cx.add_basemap(ax=_ax, source=cx.providers.CartoDB.VoyagerNoLabels, crs=4326)

ax[0].set_title("Graph clipped with a 500m buffer")
ax[1].set_title("Graph clipped by distance (500m)")
ax[2].set_title("Graphs difference")

plt.tight_layout()

In [ ]:
shortest_path_cache = {}

In [ ]:
from contextlib import suppress
from itertools import pairwise
import warnings


def subgraph_from_edge_pairs(G: nx.MultiDiGraph, edge_pairs: list[tuple[int, int]]):
    """
    Return a MultiDiGraph containing only edges whose endpoints match edge_pairs.
    - G: original MultiDiGraph (osmnx graph)
    - edge_pairs: iterable of (u, v) tuples (node ids). Treated as directed by default.
    """
    G_out = nx.MultiDiGraph()
    G_out.graph.update(G.graph)
    edge_set = set(edge_pairs)

    # add nodes that will be used (copy node attributes)
    nodes_to_add = set()
    for u, v in edge_set:
        if u in G:
            nodes_to_add.add(u)
        if v in G:
            nodes_to_add.add(v)
    for n in nodes_to_add:
        G_out.add_node(n, **G.nodes[n])

    # copy matching edges (preserve keys and attributes)
    for u, v, key, data in G.edges(keys=True, data=True):
        if (u, v) in edge_set:
            # ensure nodes exist in G_out (they should from nodes_to_add, but double-check)
            if not G_out.has_node(u):
                G_out.add_node(u, **G.nodes[u])
            if not G_out.has_node(v):
                G_out.add_node(v, **G.nodes[v])
            G_out.add_edge(u, v, key=key, **data)

    return G_out


def cut_linestring(line: LineString, distance: float) -> list[LineString]:
    if distance <= 0.0:
        return [line]
    elif distance >= 1.0:
        return [line]
    coords = list(line.coords)
    for i, p in enumerate(coords):
        pd = line.project(Point(p), normalized=True)
        if pd == distance:
            return [LineString(coords[: i + 1]), LineString(coords[i:])]
        if pd > distance:
            cp = line.interpolate(distance, normalized=True)
            return [
                LineString(coords[:i] + [(cp.x, cp.y)]),
                LineString([(cp.x, cp.y)] + coords[i:]),
            ]

    raise RuntimeError


def get_path_length(u, graph, no_cache: bool = False):
    cache_key = (center_node, u)
    if no_cache:
        path = ox.shortest_path(graph, center_node, u, weight="length")
        if not path:
            raise RuntimeError
        # path_length = sum(
        #     graph.get_edge_data(u, v)[0]["length"] for u, v in pairwise(path)
        # )
        path_length = sum(
            min(d["length"] for d in graph.get_edge_data(u, v).values())
            for u, v in pairwise(path)
        )
        return path_length
    if cache_key not in shortest_path_cache:
        path = ox.shortest_path(graph, center_node, u, weight="length")
        if not path:
            raise RuntimeError
        path_length = sum(
            min(d["length"] for d in graph.get_edge_data(u, v).values())
            for u, v in pairwise(path)
        )
        shortest_path_cache[cache_key] = path_length
    length = shortest_path_cache[cache_key]

    return length


def truncate_osmnx_graph(graph: nx.MultiDiGraph, center_point: Point, distance: float):
    subgraph_edges = None
    center_node = ox.nearest_nodes(graph, X=center_point.x, Y=center_point.y)
    subgraph = ox.truncate.truncate_graph_dist(
        graph, center_node, distance, weight="length"
    )
    with suppress(ValueError):
        subgraph_edges = ox.graph_to_gdfs(subgraph, nodes=False, edges=True)

    # find all endpoints and check their edges outside. Clip edges exactly at the distance point.
    edges_to_clip = {}
    for node in set(subgraph.nodes).union([center_node]):
        for u, v, data in graph.edges(node, keys=False, data=True):
            if v in subgraph:
                continue

            length = get_path_length(u, subgraph)
            length_left = distance - length
            if length_left > 0:
                edges_to_clip[(u, v)] = length_left
            else:
                print((u, v), length, length_left)

    print(edges_to_clip)

    pruned_edges = ox.graph_to_gdfs(
        subgraph_from_edge_pairs(graph, list(edges_to_clip.keys())),
        nodes=False,
        edges=True,
    )

    clipped_edges_geometries = []
    for (u, v), length_clip in edges_to_clip.items():
        with warnings.catch_warnings(action="ignore"):
            edge = pruned_edges.loc[(u, v)].iloc[0]
        edge_length = edge["length"]
        edge_linestring = edge["geometry"]
        interpolation_ratio = length_clip / edge_length
        clipped_edge = cut_linestring(edge_linestring, interpolation_ratio)[0]
        clipped_edges_geometries.append(clipped_edge)

    clipped_edges_gdf = gpd.GeoDataFrame(
        geometry=gpd.GeoSeries(clipped_edges_geometries, crs=4326),
    )
    clipped_edges_gdf["clipped"] = True
    if subgraph_edges is not None:
        subgraph_edges["clipped"] = False
        all_edges_gdf = gpd.pd.concat(
            [
                subgraph_edges[["geometry", "clipped"]],
                clipped_edges_gdf,
            ],
            ignore_index=True,
        )
    else:
        all_edges_gdf = clipped_edges_gdf

    return all_edges_gdf

In [ ]:
clipped_graph_with_extensions = truncate_osmnx_graph(
    graph=g_walk, center_point=center_point, distance=distance_meters
)
clipped_graph_with_extensions

In [ ]:
ax = clipped_graph_with_extensions[~clipped_graph_with_extensions["clipped"]].plot()
clipped_graph_with_extensions[clipped_graph_with_extensions["clipped"]].plot(
    ax=ax, color="orange"
)

In [ ]:
subgraph = ox.truncate.truncate_graph_dist(g_walk, center_node, 500, weight="length")
with suppress(ValueError):
    subgraph_edges = ox.graph_to_gdfs(subgraph, nodes=False, edges=True)
subgraph_edges

In [ ]:
for n in subgraph.nodes:
    length = get_path_length(n, subgraph)
    if length > 500:
        full_length = get_path_length(n, g_walk, no_cache=True)
        print(n, length, full_length)

In [ ]:
distances = nx.shortest_path_length(g_walk, source=center_node, weight="length")

# then identify every node further than dist away
distant_nodes = {k for k, v in distances.items() if v > 500}
unreachable_nodes = g_walk.nodes - distances.keys()
reachable_nodes = g_walk.nodes - (distant_nodes | unreachable_nodes)

# # make a copy to not mutate original graph object caller passed in
# G = G.copy()
# G.remove_nodes_from(distant_nodes | unreachable_nodes)

# msg = f"Truncated graph by {weight}-weighted network distance"
# utils.log(msg, level=lg.INFO)
# return G

In [ ]:
nx.shortest_path_length(g_walk, source=center_node, target=4623058006, weight="length")

In [ ]:
path = ox.shortest_path(g_walk, center_node, 4623058006, weight="length")
print(path)
for u, v in pairwise(path):
    data = g_walk.get_edge_data(u, v)
    if len(data) > 1:
        print(u, v)
        for d in data.values():
            print(d)
path_length = sum(g_walk.get_edge_data(u, v)[0]["length"] for u, v in pairwise(path))
path_length

In [ ]:
path = ox.shortest_path(g_walk, center_node, u, weight="length")
if not path:
    raise RuntimeError
path_length = sum(
    graph.get_edge_data(u, v)[0]["length"] for u, v in pairwise(path)
)
shortest_path_cache[cache_key] = path_length

In [ ]:
(4616093852, 6881284413) 505.2404393477722 -5.240439347772224
(4616093852, 3596974597) 505.2404393477722 -5.240439347772224
(6885502792, 6885502791) 501.09488423041716 -1.0948842304171649
(9855508322, 6881284413) 502.24351305259574 -2.2435130525957447
(9855508322, 9855508317) 502.24351305259574 -2.2435130525957447


In [ ]:
distances[4616093852]

In [ ]:
# 4616093852 505.2404393477722 505.2404393477722
# 4623058006 533.5932236203761 533.5932236203761
# 6885502792 501.09488423041716 501.09488423041716
# 6885502793 504.82765554802734 504.82765554802734
# 9855508322 502.24351305259574 502.24351305259574

# 4616093852 505.2404393477722 490.9224812063062
# 6885502792 501.09488423041716 496.1218942615437
# 9855508322 502.24351305259574 487.9255549111297

In [ ]:
p_sub = ox.shortest_path(subgraph, center_node, 4616093852, weight="length")
p_total = ox.shortest_path(g_walk, center_node, 4616093852, weight="length")

In [ ]:
print(p_sub)
print(p_total)

In [ ]:

subgraph_edges.loc[4623058006]

In [ ]:
subgraph_edges["u_distance"] = subgraph_edges.index.get_level_values("u").map(
    lambda u: get_path_length(u, subgraph)
)
subgraph_edges["v_distance"] = subgraph_edges.index.get_level_values("v").map(
    lambda v: get_path_length(v, subgraph)
)
subgraph_edges["closer_u"] = subgraph_edges["u_distance"] < subgraph_edges["v_distance"]
subgraph_edges["start_distance"] = np.minimum(
    subgraph_edges["u_distance"], subgraph_edges["v_distance"]
)
subgraph_edges["end_distance"] = np.maximum(
    subgraph_edges["u_distance"], subgraph_edges["v_distance"]
)
# subgraph_edges["end_distance"] = (
#     np.minimum(subgraph_edges["u_distance"], subgraph_edges["v_distance"])
#     + subgraph_edges["length"]
# )
subgraph_edges

In [ ]:
shortest_edges = subgraph_edges.sort_values("length").groupby(["u", "v"]).first()
shortest_edges

In [ ]:
subgraph_edges.sort_values('end_distance', ascending=False)

In [ ]:
duplicate_indexes = set()
for u, v, osm_id in subgraph_edges.sort_index().index:
    if u < v and (u, v, 0) in subgraph_edges.sort_index().index:
        duplicate_indexes.add((u, v))


In [ ]:
non_duplicate_edges = subgraph_edges[
    subgraph_edges.index.map(lambda idx: (idx[0], idx[1]) not in duplicate_indexes)
].copy()
non_duplicate_edges

In [ ]:
non_duplicate_edges["norm_start_dist"] = non_duplicate_edges["start_distance"] / 500
non_duplicate_edges["norm_end_dist"] = non_duplicate_edges["end_distance"] / 500
non_duplicate_edges

In [ ]:
cmap = plt.get_cmap("Oranges")

non_duplicate_edges["avg_dist"] = (
    non_duplicate_edges["norm_start_dist"] + non_duplicate_edges["norm_end_dist"]
) / 2
non_duplicate_edges["color"] = non_duplicate_edges["avg_dist"].apply(cmap)
non_duplicate_edges

In [ ]:
# from matplotlib.pyplot import get_cmap


from tqdm import tqdm


cmap = plt.get_cmap("Oranges")
non_duplicate_edges["color"] = non_duplicate_edges["avg_dist"].apply(cmap)

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
non_duplicate_edges.plot(ax=ax, color=non_duplicate_edges["color"], lw=1.5)

clipped_graph_with_extensions[clipped_graph_with_extensions["clipped"]].plot(
    ax=ax, color="white", lw=1.5, ls="--"
)

# cx.add_basemap(ax=ax, source=cx.providers.CartoDB.PositronNoLabels, crs=4326)
cx.add_basemap(ax=ax, source=cx.providers.CartoDB.DarkMatterNoLabels, crs=4326)
plt.show()

# for row in tqdm(non_duplicate_edges.to_dict(orient="records")):
#     start_val = row["norm_start_dist"]
#     end_val = row["norm_end_dist"]
#     avg_val = (start_val + end_val) / 2
#     gpd.GeoSeries([row["geometry"]], crs=4326).plot(ax=ax, color=cmap(avg_val))
#     # break

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(25, 10))

non_duplicate_edges.plot(ax=ax[0], color=non_duplicate_edges["color"], lw=1.5)
# clipped_graph_by_polygon_edges.plot(ax=ax[0], lw=1, zorder=1)
# gpd.GeoSeries([buffered_polygon_4326], crs=4326).boundary.plot(
#     ax=ax[0], lw=1.5, color="black", zorder=2, ls="--", alpha=0.4
# )

clipped_graph_with_extensions[~clipped_graph_with_extensions["clipped"]].plot(
    ax=ax[1], lw=1.5, color="white", alpha=0.2
)
clipped_graph_with_extensions[clipped_graph_with_extensions["clipped"]].plot(
    ax=ax[1], color="orange", lw=2
)

# clipped_graph_by_polygon_edges.plot(ax=ax[2], alpha=0.4, lw=1, ls="--", zorder=1)
# clipped_graph_by_distance_edges.plot(ax=ax[2], color="orange", lw=1.5, zorder=2)
clipped_graph_with_extensions[clipped_graph_with_extensions["clipped"]].plot(
    ax=ax[2], color="orange", lw=2
)


bounds = list(buffered_polygon_4326.bounds)
for _ax in ax:
    bounds_x = _ax.get_xlim()
    bounds_y = _ax.get_ylim()
    bounds[0] = min(bounds[0], bounds_x[0])
    bounds[1] = min(bounds[1], bounds_y[0])
    bounds[2] = max(bounds[2], bounds_x[1])
    bounds[3] = max(bounds[3], bounds_y[1])

for _ax in ax:
    # gpd.GeoSeries([center_point], crs=4326).plot(ax=_ax, color="black", zorder=3)
    gpd.GeoSeries([center_point], crs=4326).plot(
        ax=_ax,
        facecolor="white",
        edgecolor="black",
        zorder=3,
        markersize=100,
        lw=2,
        # edgewidth=1,
    )
    _ax.set_xlim(bounds[0], bounds[2])
    _ax.set_ylim(bounds[1], bounds[3])
    _ax.set_axis_off()
    cx.add_basemap(ax=_ax, source=cx.providers.CartoDB.DarkMatterNoLabels, crs=4326)

ax[0].set_title("Graph clipped by distance (500m)")
ax[1].set_title("Graph clipped with edges extensions")
ax[2].set_title("Graph clipped edges extensions up to 500m")

plt.tight_layout()

In [ ]:
edges_union

In [ ]:
m = clipped_graph_with_extensions.explore(
    color="white", opacity=0.2, tiles="CartoDB DarkMatter"
)

edges_union = gpd.GeoSeries([clipped_graph_with_extensions.union_all()], crs=4326)

isochrone_approx = edges_union.concave_hull(ratio=0.05)
isochrone_approx_edge = isochrone_approx.boundary.iloc[0]

isochrone_approx.explore(m=m, color="lime", style_kwds=dict(fillOpacity=0.4))
m

In [ ]:
dist = 1000

# subgraph = ox.truncate.truncate_graph_dist(g_walk, center_node, dist)
# edges = ox.graph_to_gdfs(subgraph, nodes=False, edges=True)
edges_union = gpd.GeoSeries([clipped_graph_with_extensions.union_all()], crs=4326)

# isochrone_approx = edges_union.concave_hull(ratio=0.15)
# isochrone_approx = edges_union.concave_hull(ratio=0.05)
isochrone_approx = edges_union.convex_hull
isochrone_approx_edge = isochrone_approx.boundary.iloc[0]

m = edges_union.explore(color="white", tiles="CartoDB DarkMatterNoLabels", opacity=0.8)
isochrone_approx.explore(m=m, color="lime", style_kwds=dict(fillOpacity=0.4))
# isochrone_approx_edge.explore(m=m, color="orange")

In [ ]:
def locate_farthest_intersection_point(
    center_point: Point,
    convex_hull_boundary: LineString,
    angle: float,
    raise_if_multiple: bool = False,
):
    import shapely.geometry
    import math

    ray_length = 1e5
    angle_rad = math.radians(angle)
    ray_endpoint = shapely.geometry.Point(
        center_point.x + ray_length * math.cos(angle_rad),
        center_point.y + ray_length * math.sin(angle_rad),
    )
    ray = shapely.geometry.LineString([center_point, ray_endpoint])
    intersection = convex_hull_boundary.intersection(ray)

    if intersection.is_empty:
        return None
    elif intersection.geom_type == "Point":
        return intersection
    elif intersection.geom_type in ["MultiPoint", "GeometryCollection"]:
        if raise_if_multiple:
            raise RuntimeError
        points = [geom for geom in intersection.geoms if geom.geom_type == "Point"]
        if not points:
            return None
        closest_point = max(points, key=lambda point: center_point.distance(point))
        return closest_point
    else:
        return None

In [ ]:
points = []
lines = []
for angle in np.arange(0, 360, 0.1):
    point = locate_farthest_intersection_point(
        center_point, isochrone_approx_edge, angle
    )
    if point:
        points.append(point)
        lines.append(LineString([center_point, point]))

new_boundary = Polygon(points)

m = isochrone_approx.explore(
    tiles="CartoDB DarkMatterNoLabels",
    color="white",
    opacity=0.4,
    style_kwds=dict(fillOpacity=0.1),
)
gpd.GeoSeries([new_boundary], crs=4326).explore(
    m=m, color="orange", style_kwds=dict(fillOpacity=0.2)
)
gpd.GeoSeries(points, crs=4326).explore(m=m, color="lime")
# gpd.GeoSeries(lines, crs=4326).explore(tiles="CartoDB Positron", m=m)

In [ ]:
m = gpd.GeoSeries(lines, crs=4326).explore(
    tiles="CartoDB DarkMatterNoLabels",
    style_kwds=dict(
        color="white",
        opacity=0.5,
        # style_kwds=dict(weight=0.1),
        style_function=lambda x: {
            # "color": x["properties"]["color"],
            "weight": 0.5
        }
    ),
)
# isochrone_approx.explore(color="red", style_kwds=dict(fillOpacity=0), m=m)
gpd.GeoSeries(points, crs=4326).explore(m=m, color="orange")
gpd.GeoSeries([center_point], crs=4326).explore(m=m, color="orange")
m

In [ ]:
wroclaw_buildings = convert_geometry_to_geodataframe(
    geocode_to_geometry("Wrocław"), tags_filter={"building": True}
)
wroclaw_buildings

In [ ]:
from shapely import distance
from shapely.coords import CoordinateSequence


def get_bearing(lat1, long1, lat2, long2):
    dLon = long2 - long1
    x = np.cos(np.radians(lat2)) * np.sin(np.radians(dLon))
    y = np.cos(np.radians(lat1)) * np.sin(np.radians(lat2)) - np.sin(
        np.radians(lat1)
    ) * np.cos(np.radians(lat2)) * np.cos(np.radians(dLon))
    brng = np.arctan2(x, y)
    brng = np.degrees(brng)

    return brng


def get_angle(point1: Point, point2: Point):
    rads = np.arctan2(point2.y - point1.y, point2.x - point1.x)
    return np.degrees(rads)


def transform_point(
    point: Point, center_point: Point, isochrone_boundary: Polygon
) -> Point:
    angle = get_angle(center_point, point)
    intersection_point = locate_farthest_intersection_point(
        center_point, isochrone_boundary.exterior, angle
    )

    distance_from_isochrone_boundary = distance(center_point, intersection_point)
    distance_from_current_point = distance(center_point, point)
    distance_ratio = min(
        1, distance_from_current_point / distance_from_isochrone_boundary
    )

    length = distance_ratio

    angle_rad = np.radians(angle)

    new_point = Point(length * np.cos(angle_rad), length * np.sin(angle_rad))

    return new_point


def transform_coords(
    coords: CoordinateSequence,
    center_point: Point,
    isochrone_boundary: Polygon,
) -> list[Point]:
    result = []
    coords_list = list(coords)
    for (x1, y1), (x2, y2) in pairwise(coords_list):
        result.append(transform_point(Point(x1, y1), center_point, isochrone_boundary))
        ls = LineString([(x1, y1), (x2, y2)])
        step = 1 / 8
        for idx in np.arange(0, 1, step):
            result.append(
                transform_point(
                    ls.interpolate(idx, normalized=True),
                    center_point,
                    isochrone_boundary,
                )
            )


    result.append(
        transform_point(Point(coords_list[-1]), center_point, isochrone_boundary)
    )
    return result


def transform_geometries(
    gs: gpd.GeoSeries,
    center_point: Point,
    isochrone_boundary: Polygon,
):
    geoms = []
    for geometry in gs:
        if isinstance(geometry, Polygon):
            transformed_ex = transform_coords(
                geometry.exterior.coords,
                center_point,
                isochrone_boundary,
            )
            transformed_ins = [
                transform_coords(
                    interior.coords, center_point, isochrone_boundary
                )
                for interior in geometry.interiors
            ]
            geoms.append(Polygon(transformed_ex, transformed_ins))
        elif isinstance(geometry, LineString):
            transformed_coords = transform_coords(
                geometry.coords, center_point, isochrone_boundary
            )
            geoms.append(LineString(transformed_coords))

    return gpd.GeoSeries(geoms)


In [ ]:
from shapely import LinearRing, polygonize, unary_union


all_points = gpd.GeoSeries(
    clipped_graph_with_extensions.get_coordinates(ignore_index=True).apply(
        lambda row: Point(row["x"], row["y"]), axis=1
    ),
    crs=4326,
)
points_outside_boundary = gpd.GeoSeries([], crs=4326)
end_points = clipped_graph_with_extensions[
    clipped_graph_with_extensions["clipped"]
].geometry.apply(lambda ls: Point(ls.coords[-1]))

polygonized_edges = gpd.GeoSeries(unary_union(polygonize([edges_union])), crs=4326)
sorted_end_points = gpd.GeoSeries(
    sorted(end_points, key=lambda pt: get_angle(center_point, pt)), crs=4326
).drop_duplicates()
isochrone_boundary_old = Polygon(
    sorted_end_points
)  # <- show in blogpost why changed to polygonize with 1000m example
isochrone_boundary = unary_union(
    polygonize(edges_union.union(LinearRing(sorted_end_points)).to_list()).geoms
)

In [ ]:
polygonized_edges

In [ ]:
m = gpd.GeoSeries([isochrone_boundary_old], crs=4326).explore(
    tiles="CartoDB DarkMatterNoLabels",
    color="white",
    opacity=0.8,
    style_kwds=dict(fillOpacity=0.2),
)

clipped_graph_with_extensions[clipped_graph_with_extensions["clipped"]].explore(
    m=m, color="orange"
)

gpd.GeoSeries(
    [
        clipped_graph_with_extensions[
            ~clipped_graph_with_extensions["clipped"]
        ].union_all()
    ],
    crs=4326,
).explore(color="orange", opacity=0.2, m=m)
sorted_end_points.explore(color="white", m=m)

# isochrone_approx.explore(color="red", style_kwds=dict(fillOpacity=0), m=m)
# gpd.GeoSeries(points, crs=4326).explore(m=m, color="orange")
# gpd.GeoSeries([center_point], crs=4326).explore(m=m, color="orange")
m

In [ ]:
m = gpd.GeoSeries([isochrone_boundary_old], crs=4326).boundary.explore(
    tiles="CartoDB DarkMatterNoLabels",
    color="white",
    opacity=0.8,
    style_kwds=dict(fillOpacity=0.2),
)

# clipped_graph_with_extensions[clipped_graph_with_extensions["clipped"]].explore(
#     m=m, color="orange"
# )

gpd.GeoSeries(
    [
        clipped_graph_with_extensions[
            ~clipped_graph_with_extensions["clipped"]
        ].union_all()
    ],
    crs=4326,
).explore(color="white", opacity=0.2, m=m)
# sorted_end_points.explore(color="white", m=m)
polygonized_edges.explore(
    color="royalblue",
    m=m,
    opacity=0.8,
    style_kwds=dict(fillOpacity=0.2),
)

gpd.GeoSeries([isochrone_boundary], crs=4326).boundary.explore(
    color="orange", m=m
)

# isochrone_approx.explore(color="red", style_kwds=dict(fillOpacity=0), m=m)
# gpd.GeoSeries(points, crs=4326).explore(m=m, color="orange")
# gpd.GeoSeries([center_point], crs=4326).explore(m=m, color="orange")
m

In [ ]:
from shapely import LinearRing, polygonize, unary_union


def generate_isochone_boundary(
    clipped_edges: gpd.GeoDataFrame,
    edges_union: gpd.GeoSeries,
    previous_boundary: Polygon | None = None,
    steps_between_edges: int = 8,
) -> Polygon:
    all_points = gpd.GeoSeries(
        clipped_edges.get_coordinates(ignore_index=True).apply(
            lambda row: Point(row["x"], row["y"]), axis=1
        ),
        crs=4326,
    )
    points_outside_boundary = gpd.GeoSeries([], crs=4326)
    end_points = clipped_edges[clipped_edges["clipped"]].geometry.apply(
        lambda ls: Point(ls.coords[-1])
    )

    sorted_end_points = gpd.GeoSeries(
        sorted(end_points, key=lambda pt: get_angle(center_point, pt)), crs=4326
    ).drop_duplicates()
    # isochrone_boundary = Polygon(sorted_end_points) # <- show in blogpost why changed to polygonize with 1000m example
    isochrone_boundary = unary_union(
        polygonize(edges_union.union(LinearRing(sorted_end_points)).to_list()).geoms
    )

    # if previous_boundary is not None:
    #     isochrone_boundary = isochrone_boundary.union(previous_boundary)

    points_outside_boundary = all_points[~all_points.intersects(isochrone_boundary)]
    if len(points_outside_boundary) > 0:
        end_points = gpd.pd.concat([end_points, points_outside_boundary])
        sorted_end_points = gpd.GeoSeries(
            sorted(end_points, key=lambda pt: get_angle(center_point, pt)), crs=4326
        ).drop_duplicates()
        isochrone_boundary = unary_union(
            polygonize(edges_union.union(LinearRing(sorted_end_points)).to_list()).geoms
        )

    # # add points in between
    # coords = list(isochrone_boundary.exterior.coords)
    # step = 1 / steps_between_edges
    # interpolated_boundary = []
    # for pt1, pt2 in pairwise(coords):
    #     ls = LineString([pt1, pt2])
    #     # print(ls)
    #     step = 1 / steps_between_edges
    #     for idx in np.arange(0, 1, step):
    #         interpolated_boundary.append(ls.interpolate(idx, normalized=True))
    # interpolated_boundary.append(Point(coords[0]))

    # isochrone_boundary = Polygon(interpolated_boundary)

    if previous_boundary is not None:
        isochrone_boundary = isochrone_boundary.union(previous_boundary)

    if isochrone_boundary.geom_type == "MultiPolygon":
        print(isochrone_boundary)
        print(sorted(isochrone_boundary.geoms, key=lambda g: g.area, reverse=True))

        isochrone_boundary = sorted(
            isochrone_boundary.geoms, key=lambda g: g.area, reverse=True
        )[0]
        print(isochrone_boundary)

    return isochrone_boundary


def generate_isochrone_data(
    distance_m: float, previous_boundary: Polygon | None = None, transform: bool = True
):
    clipped_edges = truncate_osmnx_graph(g_walk, center_point, distance_m)
    edges_union = gpd.GeoSeries([clipped_edges.union_all()], crs=4326)

    isochrone_boundary = generate_isochone_boundary(
        clipped_edges, edges_union, previous_boundary
    )

    clipped_buildings = wroclaw_buildings.clip(isochrone_boundary).explode()
    clipped_buildings = clipped_buildings[clipped_buildings.geom_type == "Polygon"]

    if not transform:
        return isochrone_boundary, clipped_buildings, edges_union

    transformed_buildings = transform_geometries(
        clipped_buildings.geometry, center_point, isochrone_boundary
    )
    transformed_edges = transform_geometries(
        edges_union.explode(), center_point, isochrone_boundary
    )

    return (
        isochrone_boundary,
        clipped_buildings,
        edges_union,
        transformed_buildings,
        transformed_edges,
    )

In [ ]:
def plot_data(
    isochrone_boundary, buildings, edges, transformed_buildings, transformed_edges, distance
):
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))

    ###

    gpd.GeoSeries([isochrone_boundary], crs=4326).exterior.plot(ax=ax1, color="C3", zorder=4)
    edges.plot(ax=ax1, color="C0", lw=1, zorder=1)
    # edges.plot(ax=ax1, color="C0", lw=1, zorder=1, alpha=0.2)
    if len(buildings) > 0:
        buildings.plot(ax=ax1, color="C1", alpha=0.4, zorder=2)
        buildings.boundary.plot(ax=ax1, color="C1", lw=1, zorder=3)
    gpd.GeoSeries([center_point], crs=4326).plot(ax=ax1, color="C2", zorder=4)

    ###

    gpd.GeoSeries([Point(0, 0).buffer(1)]).exterior.plot(ax=ax2, color="C3", zorder=4)
    transformed_edges.plot(ax=ax2, color="C0", lw=1, zorder=1)
    # transformed_edges.plot(ax=ax2, color="C0", lw=1, zorder=1, alpha=0.2)
    if len(transformed_buildings) > 0:
        transformed_buildings.plot(ax=ax2, color="C1", alpha=0.4, zorder=2)
        transformed_buildings.boundary.plot(ax=ax2, color="C1", lw=1, zorder=3)
    gpd.GeoSeries([Point(0, 0)]).plot(ax=ax2, color="C2", zorder=4)

    ###

    ax1.set_axis_off()
    ax2.set_axis_off()

    with warnings.catch_warnings(action="ignore"):
        cx.add_basemap(ax1, source="CartoDB VoyagerNoLabels", crs=4326)

    ax1.set_title(f"Geographic isochrone (distance {distance} meters)")
    ax2.set_title(f"Chronographic isochrone (distance {distance} meters)")

    return fig

    # plt.tight_layout()

    # plt.show()

In [ ]:
wroclaw_buildings

In [ ]:
clipped_buildings = wroclaw_buildings.clip(isochrone_boundary).explode()
clipped_buildings = clipped_buildings[clipped_buildings.geom_type == 'Polygon']
clipped_buildings

In [ ]:
m = edges_union.explore(color="black", tiles="CartoDB VoyagerNoLabels", opacity=0.6)
clipped_buildings.explore(m=m)
gpd.GeoSeries([isochrone_boundary], crs=4326).boundary.explore(
    color="black", m=m, dash_array="10", opacity=0.8
)

In [ ]:
(
    test_isochrone_boundary,
    test_clipped_buildings,
    test_edges_union,
    test_transformed_buildings,
    test_transformed_edges,
) = generate_isochrone_data(500)

In [ ]:
plot_data(
    isochrone_boundary=test_isochrone_boundary,
    buildings=test_clipped_buildings,
    edges=test_edges_union,
    transformed_buildings=test_transformed_buildings,
    transformed_edges=test_transformed_edges,
    distance=500,
)

In [ ]:
(
    test_isochrone_boundary,
    test_clipped_buildings,
    test_edges_union,
    test_transformed_buildings,
    test_transformed_edges,
) = generate_isochrone_data(100)

In [ ]:
idx = test_clipped_buildings.index.get_loc("way/101127025")

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))


def get_point_transformation(
    point: Point, center_point: Point, isochrone_boundary: Polygon
):
    angle = get_angle(center_point, point)
    intersection_point = locate_farthest_intersection_point(
        center_point, isochrone_boundary.exterior, angle
    )

    distance_from_isochrone_boundary = distance(center_point, intersection_point)
    distance_from_current_point = distance(center_point, point)
    distance_ratio = min(
        1, distance_from_current_point / distance_from_isochrone_boundary
    )

    length = distance_ratio

    angle_rad = np.radians(angle)

    new_point = Point(length * np.cos(angle_rad), length * np.sin(angle_rad))

    return (
        intersection_point,
        distance_from_isochrone_boundary,
        distance_from_current_point,
        distance_ratio,
        angle_rad,
        new_point,
    )


###

tcb = test_clipped_buildings.iloc[[idx]]

gpd.GeoSeries([test_isochrone_boundary], crs=4326).exterior.plot(
    ax=ax1, color="C3", zorder=4
)
# edges.plot(ax=ax1, color="C0", lw=1, zorder=1)
# test_edges_union.plot(ax=ax1, color="C0", lw=1, zorder=1, alpha=0.2)
if len(test_clipped_buildings) > 0:
    # tcb.plot(ax=ax1, color="C1", alpha=0.4, zorder=2)
    # tcb.boundary.plot(ax=ax1, color="C1", lw=1, zorder=3)
    tcb.boundary.plot(ax=ax1, color="black", lw=1, zorder=3)
gpd.GeoSeries([center_point], crs=4326).plot(ax=ax1, color="C2", zorder=4)

###

ttb = test_transformed_buildings.iloc[[idx]]

gpd.GeoSeries([Point(0, 0).buffer(1)]).exterior.plot(ax=ax2, color="C3", zorder=4)
# transformed_edges.plot(ax=ax2, color="C0", lw=1, zorder=1)
# test_transformed_edges.plot(ax=ax2, color="C0", lw=1, zorder=1, alpha=0.2)
if len(test_transformed_buildings) > 0:
    # ttb.plot(ax=ax2, color="C1", alpha=0.4, zorder=2)
    # ttb.boundary.plot(ax=ax2, color="C1", lw=1, zorder=3)
    ttb.boundary.plot(ax=ax2, color="black", lw=1, zorder=3)
gpd.GeoSeries([Point(0, 0)]).plot(ax=ax2, color="C2", zorder=4)

###

ax1.set_axis_off()
ax2.set_axis_off()

with warnings.catch_warnings(action="ignore"):
    # cx.add_basemap(ax1, source="CartoDB VoyagerNoLabels", crs=4326)
    cx.add_basemap(ax1, source="CartoDB PositronNoLabels", crs=4326)

ax1.set_title(f"Geographic isochrone (distance {100} meters)")
ax2.set_title(f"Chronographic isochrone (distance {100} meters)")

radial_center_point = Point(0, 0)
for b in tcb.geometry:
    print(b)
    coords_list = list(b.exterior.coords)
    for idx, ((x1, y1), (x2, y2)) in enumerate(pairwise(coords_list)):
        pt = Point(x1, y1)
        (
            intersection_point,
            distance_from_isochrone_boundary,
            distance_from_current_point,
            distance_ratio,
            angle_rad,
            new_point,
        ) = get_point_transformation(
            pt,
            center_point=center_point,
            isochrone_boundary=test_isochrone_boundary,
        )
        print(intersection_point)

        line_to_point = LineString((center_point, pt))
        line_to_boundary = LineString((pt, intersection_point))

        gpd.GeoSeries([line_to_point], crs=4326).plot(ax=ax1, color="C1")
        gpd.GeoSeries([line_to_boundary], crs=4326).plot(
            ax=ax1, color="black", ls="--", alpha=0.4
        )

        if idx in (1, 3):
            middle_x = (center_point.x + pt.x) / 2
            middle_y = (center_point.y + pt.y) / 2
            if idx == 1:
                middle_y -= 0.00002
            ax1.text(
                middle_x,
                middle_y,
                s=f"{100 * distance_ratio:.2f}%",
                verticalalignment="bottom" if idx == 3 else "top",
                horizontalalignment="right" if idx == 3 else "center",
            )
            middle_x = (intersection_point.x + pt.x) / 2
            middle_y = (intersection_point.y + pt.y) / 2
            if idx == 1:
                middle_y -= 0.00002
            ax1.text(
                middle_x,
                middle_y,
                s=f"{100 - 100 * distance_ratio:.2f}%",
                verticalalignment="bottom" if idx == 3 else "top",
                horizontalalignment="right" if idx == 3 else "center",
            )

        line_to_point = LineString((radial_center_point, new_point))
        radial_boundary_point = Point(np.cos(angle_rad), np.sin(angle_rad))
        line_to_boundary = LineString((new_point, radial_boundary_point))

        gpd.GeoSeries([line_to_point], crs=4326).plot(ax=ax2, color="C1")
        gpd.GeoSeries([line_to_boundary], crs=4326).plot(
            ax=ax2, color="black", ls="--", alpha=0.4
        )

        if idx in (1, 3):
            middle_x = (radial_center_point.x + new_point.x) / 2
            middle_y = (radial_center_point.y + new_point.y) / 2
            if idx == 1:
                middle_y -= 0.015
            ax2.text(
                middle_x,
                middle_y,
                s=f"{100 * distance_ratio:.2f}%",
                verticalalignment="bottom" if idx == 3 else "top",
                horizontalalignment="right" if idx == 3 else "center",
            )
            middle_x = (radial_boundary_point.x + new_point.x) / 2
            middle_y = (radial_boundary_point.y + new_point.y) / 2
            if idx == 1:
                middle_y -= 0.015
            ax2.text(
                middle_x,
                middle_y,
                s=f"{100 - 100 * distance_ratio:.2f}%",
                verticalalignment="bottom" if idx == 3 else "top",
                horizontalalignment="right" if idx == 3 else "center",
            )

# plot_data(
#     isochrone_boundary=test_isochrone_boundary,
#     buildings=test_clipped_buildings.iloc[[idx]],
#     edges=test_edges_union,
#     transformed_buildings=test_transformed_buildings.iloc[[idx]],
#     transformed_edges=test_transformed_edges,
#     distance=100,
# )
plt.show()

In [ ]:
(
    test_isochrone_boundary,
    test_clipped_buildings,
    test_edges_union,
    test_transformed_buildings,
    test_transformed_edges,
) = generate_isochrone_data(20)

In [ ]:
plot_data(
    isochrone_boundary=test_isochrone_boundary,
    buildings=test_clipped_buildings,
    edges=test_edges_union,
    transformed_buildings=test_transformed_buildings,
    transformed_edges=test_transformed_edges,
    distance=20,
)

In [ ]:
# interpolate between isochrones

from shapely import distance
from shapely.coords import CoordinateSequence


def transform_point_between_isochrones(
    point: Point,
    center_point: Point,
    isochrone_boundary_far: Polygon,
    isochrone_boundary_close: Polygon | None,
    vector_start=0.0,
    vector_length=1.0,
) -> Point:
    angle = get_angle(center_point, point)
    intersection_point = locate_farthest_intersection_point(
        center_point, isochrone_boundary_far.exterior, angle
    )
    distance_from_far_isochrone_boundary = distance(center_point, intersection_point)
    distance_from_current_point = distance(center_point, point)

    if isochrone_boundary_close is None:
        distance_ratio = min(
            1, distance_from_current_point / distance_from_far_isochrone_boundary
        )
    else:
        close_intersection_point = locate_farthest_intersection_point(
            center_point, isochrone_boundary_close.exterior, angle
        )
        distance_from_close_isochrone_boundary = distance(
            center_point, close_intersection_point
        )
        distance_ratio = min(
            1,
            (distance_from_current_point - distance_from_close_isochrone_boundary)
            / (
                distance_from_far_isochrone_boundary
                - distance_from_close_isochrone_boundary
            ),
        )

    length = vector_length * distance_ratio + vector_start

    angle_rad = np.radians(angle)

    new_point = Point(length * np.cos(angle_rad), length * np.sin(angle_rad))

    return new_point


def transform_coords_between_isochrones(
    coords: CoordinateSequence,
    center_point: Point,
    isochrone_boundary_far: Polygon,
    isochrone_boundary_close: Polygon | None,
    vector_start=0.0,
    vector_length=1.0,
) -> list[Point]:
    return [
        transform_point_between_isochrones(
            Point(x, y),
            center_point,
            isochrone_boundary_far,
            isochrone_boundary_close,
            vector_start,
            vector_length,
        )
        for x, y in coords
    ]


def transform_geometries_between_isochrones(
    gs: gpd.GeoSeries,
    center_point: Point,
    isochrone_boundary_far: Polygon,
    isochrone_boundary_close: Polygon | None,
    vector_start=0.0,
    vector_length=1.0,
):
    geoms = []
    for geometry in gs:
        if isinstance(geometry, Polygon):
            transformed_ex = transform_coords_between_isochrones(
                geometry.exterior.coords,
                center_point,
                isochrone_boundary_far,
                isochrone_boundary_close,
                vector_start,
                vector_length,
            )
            transformed_ins = [
                transform_coords_between_isochrones(
                    interior.coords,
                    center_point,
                    isochrone_boundary_far,
                    isochrone_boundary_close,
                    vector_start,
                    vector_length,
                )
                for interior in geometry.interiors
            ]
            geoms.append(Polygon(transformed_ex, transformed_ins))
        elif isinstance(geometry, LineString):
            transformed_coords = transform_coords_between_isochrones(
                geometry.coords,
                center_point,
                isochrone_boundary_far,
                isochrone_boundary_close,
                vector_start,
                vector_length,
            )
            geoms.append(LineString(transformed_coords))

    return gpd.GeoSeries(geoms)

In [ ]:
distances = [
    # 200,
    # 400,
    # 600,
    # 800,
    # 1000,
    # 1200,
    # 1400,
    # 1600,
    # 1800,
    # 2000,
    2200,
    2400,
    2600,
    2800,
    3000,
    3200,
    3400,
    3600,
    3800,
    4000,
]
# distances = [500, 1000, 1500, 2000]
# isochrones = []
# buildings = []
# edges = []

for distance_m in distances:
    _isochrone_boundary, _clipped_buildings, _edges_union = generate_isochrone_data(
        distance_m, transform=False
    )
    isochrones.append(_isochrone_boundary)
    buildings.append(_clipped_buildings)
    edges.append(_edges_union)

In [ ]:
m = gpd.GeoSeries(
    isochrones, crs=4326
).boundary.explore(tiles="CartoDB Voyager")
# clipped_buildings_200_without_100.explore(m=m, color="red")
# clipped_buildings_100.explore(m=m, color="orange")
m

In [ ]:
gpd.GeoSeries([isochrone_close], crs=4326).explode(
    ignore_index=True
).reset_index().explore("index")

In [ ]:
transformed_buildings = []
transformed_edges = []

for _i in range(len(isochrones) - 1, -1, -1):
    print(_i)
    isochrone_far = isochrones[_i]
    isochrone_close = isochrones[_i - 1] if _i > 0 else None

    clipped_buildings = buildings[_i].explode()
    clipped_edges = edges[_i].explode()

    # print(
    #     clipped_buildings.difference(isochrone_close)
    #     if isochrone_close is not None
    #     else clipped_buildings
    # )

    # (
    #     clipped_edges.difference(isochrone_close)
    #     if isochrone_close is not None
    #     else clipped_edges
    # ).plot()

    _transformed_buildings = transform_geometries_between_isochrones(
        clipped_buildings.difference(isochrone_close).explode()
        if isochrone_close is not None
        else clipped_buildings.geometry,
        center_point=center_point,
        isochrone_boundary_far=isochrone_far,
        isochrone_boundary_close=isochrone_close,
        vector_start=_i,
    )

    # print(_transformed_buildings)

    _transformed_edges = transform_geometries_between_isochrones(
        clipped_edges.difference(isochrone_close).explode()
        if isochrone_close is not None
        else clipped_edges.geometry,
        center_point=center_point,
        isochrone_boundary_far=isochrone_far,
        isochrone_boundary_close=isochrone_close,
        vector_start=_i,
    )
    # print(_transformed_edges)

    transformed_buildings.append(_transformed_buildings)
    transformed_edges.append(_transformed_edges)

# for _isochrone_boundary, _clipped_buildings, _edges_union in enumerate(zip(isochrones, buildings, edges):


In [ ]:
(
    test_isochrone_boundary,
    test_clipped_buildings,
    test_edges_union,
    test_transformed_buildings,
    test_transformed_edges,
) = generate_isochrone_data(500)

In [ ]:
plot_data(
    isochrone_boundary=test_isochrone_boundary,
    buildings=test_clipped_buildings,
    edges=test_edges_union,
    transformed_buildings=test_transformed_buildings,
    transformed_edges=test_transformed_edges,
    distance=500,
)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
cmap = plt.get_cmap("turbo")

for _i in range(len(isochrones) - 1, -1, -1):
    print(_i)
    color_idx = _i / (len(isochrones) - 1)
    color_val = cmap(color_idx)
    isochrone_far = isochrones[_i]
    isochrone_close = isochrones[_i - 1] if _i > 0 else None

    if isochrone_close is not None:
        clipped_buildings = buildings[_i].difference(isochrone_close)
        clipped_edges = edges[_i].difference(isochrone_close)
    else:
        clipped_buildings = buildings[_i]
        clipped_edges = edges[_i]

    tb = transformed_buildings[-_i]
    te = transformed_edges[-_i]

    ###

    gpd.GeoSeries([isochrone_far], crs=4326).exterior.plot(
        ax=ax1, color="black", lw=1, ls="--", alpha=0.5, zorder=4
    )
    # clipped_edges.plot(ax=ax1, color="C0", lw=1, zorder=1)
    clipped_edges.plot(ax=ax1, color="C0", lw=1, zorder=1, alpha=0.6)
    # edges.plot(ax=ax1, color="C0", lw=1, zorder=1, alpha=0.6)
    if len(buildings) > 0:
        clipped_buildings.plot(ax=ax1, color="C1", alpha=0.4, zorder=2)
        # clipped_buildings.boundary.plot(ax=ax1, color="C1", lw=1, zorder=3)

    ###

    gpd.GeoSeries([Point(0, 0).buffer(_i + 1)]).exterior.plot(
        ax=ax2, color="black", zorder=4, lw=1, ls="--", alpha=0.5
    )
    # ax2.text(
    #     x=0,
    #     y=-(_i + 1) - 0.08,
    #     s=f"{(_i + 1) * 500}m",
    #     va="top",
    #     ha="center",
    #     zorder=5,
    #     bbox=dict(facecolor="white", edgecolor="black", alpha=0.6),
    # )
    # te.plot(ax=ax2, color="C0", lw=1, zorder=1)
    # te.plot(ax=ax2, color="C0", lw=1, zorder=1, alpha=0.6)
    te.plot(ax=ax2, color=color_val, lw=1, zorder=1, alpha=0.6)
    # transformed_edges.plot(ax=ax2, color="C0", lw=1, zorder=1, alpha=0.2)
    if len(transformed_buildings) > 0:
        # tb.plot(ax=ax2, color="C1", alpha=0.4, zorder=2)
        tb.plot(ax=ax2, color=color_val, alpha=0.4, zorder=2)
        # tb.boundary.plot(ax=ax2, color="C1", lw=1, zorder=3)

    # break

gpd.GeoSeries([Point(0, 0).buffer(len(isochrones))]).exterior.plot(
    ax=ax2, color="black", zorder=4
)
gpd.GeoSeries([isochrones[-1]], crs=4326).exterior.plot(ax=ax1, color="black", zorder=4)
# gpd.GeoSeries([Point(0, 0)]).plot(ax=ax2, color="C1", zorder=4)
gpd.GeoSeries([Point(0, 0)]).plot(
    ax=ax2,
    facecolor="white",
    edgecolor="black",
    zorder=4,
    markersize=50,
    lw=1,
    # edgewidth=1,
)
gpd.GeoSeries([center_point], crs=4326).plot(
    ax=ax1,
    facecolor="white",
    edgecolor="black",
    zorder=4,
    markersize=50,
    lw=1,
    # edgewidth=1,
)

###

ax1.set_axis_off()
ax2.set_axis_off()

with warnings.catch_warnings(action="ignore"):
    cx.add_basemap(ax1, source="CartoDB VoyagerNoLabels", crs=4326)

ax1.set_title(f"Geographic isochrones (distances 0 to 2000 meters every 200 meters)")
ax2.set_title(f"Chronographic isochrones (distances 0 to 2000 meters every 200 meters)")

plt.show()

In [ ]:
full_transformed_buildings = transform_geometries_between_isochrones(
    buildings[-1].geometry,
    center_point=center_point,
    isochrone_boundary_far=isochrones[-1],
    isochrone_boundary_close=None,
    vector_start=0,
    vector_length=len(isochrones),
)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))

# ax = None
for _b in transformed_buildings:
    _b.boundary.plot(ax=ax, alpha=0.6)

full_transformed_buildings.boundary.plot(ax=ax, alpha=0.6, color="orange")

gpd.GeoSeries(
    [Point(0, 0).buffer(_i + 1) for _i in range(len(isochrones))]
).boundary.plot(ax=ax)

ax.set_axis_off()
ax.set_title("Difference between full transformation and partial transformation between isochrones")

plt.tight_layout()

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))

# ax = None
for _b in transformed_buildings:
    _b.plot(ax=ax, alpha=0.4, color="C0")

full_transformed_buildings.plot(ax=ax, alpha=0.4, color="C1")

gpd.GeoSeries(
    [Point(0, 0).buffer(_i + 1) for _i in range(len(isochrones))]
).boundary.plot(ax=ax)

ax.set_axis_off()
ax.set_title(
    "Difference between full transformation and partial transformation between isochrones (distances 500, 1000, 1500, 2000 meters)"
)

from matplotlib.patches import Patch

legend_elements = [
    Patch(facecolor="C1", alpha=0.8),
    Patch(facecolor="C0", alpha=0.8),
]
ax.legend(legend_elements, ["Single transformation from 0 to 2000 meters", "Partial transformation every 500 meters"])

plt.tight_layout()

plt.show()